In [1]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

import pandas as pd
import os
import torch
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import soundfile as sf
import os
# Use old or new vocab? 
#os.system("cp vocab_300_with_numbers.json vocab.json")



tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
        "/home/peterr/macocu/task8/", unk_token="[UNK]", 
        pad_token="[PAD]", word_delimiter_token=" ")
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)
model = Wav2Vec2ForCTC.from_pretrained("/home/peterr/macocu/task8/31_normalised_220k/checkpoint-6500").cuda()

def transcribe(path):
    speech, sample_rate = sf.read(path)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
import datasets
from datasets import load_dataset, load_metric, Audio, load_from_disk
test = load_from_disk("test_mapped")
train = load_from_disk("train220k_mapped")

In [3]:
speech, sample_rate = test[0]["input_values"], 16000
input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()

logits = model(input_values).logits

predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0]).lower()
transcription

'zahtjevima europske komisije u odnosu na direkdirektivu europskog parlamenta i vijeća od 5 srpnja 206 godine zahvaljujem se na podršci svima koji su raspravljali i na samomekraju vezano uz ratifikaciju istambolske konvencije'

In [4]:
labs = test[0]["labels"]
processor.decode(labs).lower()

'zahtjevima europske komisije u odnosu na direktivu europskog parlamenta i vijeća od 5. srpnja 206. godine zahvaljujem se na podršci svima koji su raspravljali i na samome kraju vezano uz ratifikaciju istambulske konvencije'

In [5]:
def decode_labels(labels):
    return processor.decode(labels).lower()
def transcribe(inputs):
    speech = inputs
    input_values = processor(speech, sampling_rate=16000, return_tensors="pt").input_values.cuda()

    logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0]).lower()
    return transcription


In [6]:
import numpy as np
max_i = len(train)
randoms = np.random.randint(0, max_i, size=10, dtype=int)

# Transcribing train data:
transcriptions = list(map(transcribe, [train[int(i)]["input_values"] for i in randoms]))
labels = list(map(decode_labels, [train[int(i)]["labels"] for i in randoms]))

for t, l in zip(transcriptions, labels):
    print("transcript:\t".upper(), t)
    print("labels:\t\t".upper(), l)
    print("\n")

TRANSCRIPT:	 poglavito s područja kojeg i sam dolazim a koje su uložile značajna sredstva u mjere zaštite okoliša poput ugradnje vlastitog pročistača za otpadne vode a i dalje plaćaju iznimno visoke naknade za onečišćenje pa evo konkretno pitanje hoćete li nešto poduzeti po tom pitanju
LABELS:		 poglavito s područja kojeg i sam dolazim a koje su uložile značajna sredstva u mjere zaštite okoliša poput ugradnje vlastitog pročistača za otpadne vode a i dalje plaćaju iznimno visoke naknade za onečišćenje pa evo konkretno pitanje hoćete li nešto poduzeti po tom pitanju


TRANSCRIPT:	 dječji doplatak ne daje samo povećanje iznosa nego je povećan obuhvat djece koja imaju pravo na dječji doplatak povećane su rodiljne naknade a u obrazovanju izmjenom zakona i reforma je u učionici strukovno obrazovanje osigurali
LABELS:		 dječji doplatak ne daje samo povećanje iznosa nego je povećan obuhvat djece koja imaju pravo na dječji doplatak povećane su rodiljne naknade a u obrazovanju izmjenom zakona i 

In [7]:
transcriptions = list(map(transcribe, test["input_values"]))
labels = list(map(decode_labels, test["labels"]))

import pandas as pd
pd.DataFrame(data={
    "machine_generated_transcripts": transcriptions,
    "original_transcript": labels
}).to_csv("34_model31_220k_performance.csv", index=False)

# Transcribing the entire train dataset and identifying weird instances

In [21]:

M = 10000
# Transcribing train data:
transcriptions = [transcribe(train[i]["input_values"]) for i in range(M)]
labels = [decode_labels(train[i]["labels"]) for i in range(M)]


In [22]:
import pandas as pd

df = pd.DataFrame(
    data={
        "labels": labels,
        "transcripts": transcriptions
    }
)

cers = []
for i, row in df.iterrows():
    cers.append(
        cer_metric.compute(predictions=[row["transcripts"]], references=[row["labels"]])
    )
df["cer"] = cers

df.sort_values(by="cer", ascending=False)

,labels,transcripts,cer
625,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
4260,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
7269,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
7568,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
2747,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
...,...,...,...
3538,stotinama milijuna kuna dakle kao takvi jer ne...,stotinama milijuna kuna dakle kao takvi jer ne...,0.000000
3540,dakle imali smo u vidu tijela državne uprave k...,dakle imali smo u vidu tijela državne uprave k...,0.000000
3541,bez obzira što 6 istarskih gradova u hrvatskih...,bez obzira što 6 istarskih gradova u hrvatskih...,0.000000
3542,vi znate da se prvo pristupilo smanjenju 10 po...,vi znate da se prvo pristupilo smanjenju 10 po...,0.000000


In [24]:
df.sort_values(by="cer", ascending=False).head(10)

,labels,transcripts,cer
625,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
4260,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
7269,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
7568,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
2747,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
929,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
4562,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
9063,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
3353,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
6069,hvala lijepo poštovani kolega bulj kada ste sp...,hala lijep poštovani kolega bulj kad s spomenu...,0.165517
